In [1]:
from IPython.core.display import HTML
HTML("<style>.container { width:95% !important; }</style>")

# Lecture 7, direct methods for constrained optimization

## Structure of optimization methods

Typically

* Constraint handling **converts** the problem to (a series of) unconstrained problems
* In unconstrained optimization a **search direction** is determined at each iteration
* The best solution in the search direction is found with **line search**

![](images/OptimizationStruc1.jpg)

## Classification of the constraint optimization methods

* **Indirect methods:** the constrained problem is converted into a sequence of unconstrained problems whose solutions will approach to the solution of the constrained problem, the intermediate solutions need not to be feasible 

* **Direct methods:** the constraints are taking into account explicitly, intermediate solutions are feasible

# Direct methods for constrained optimization

Direct methods for constrained optimization are also known as *methods of feasible directions*

### Idea

* in a point $x_k$, generate a feasible search direction where objective function value can be improved
* use line search to get $x_{k+1}$ 

### Methods differ in

* how to choose a feasible direction and
* what is assumed from the constraints (linear/nonlinear, equality/inequality)

## Feasible descent directions

Let $S\subset \mathbb R^n$ ($S\neq \emptyset$) and $x^*\in S$. 

**Definition:** The set
$$ F = \{d\in \mathbb R^n: d\neq0,x^*+\alpha d\in S \text{ for all } \alpha\in (0,\delta) \text{ for some } \delta>0\}$$

is called the cone of feasible directions of $S$ in $x^*$.

**Definition:** The set 
$$ D = \{d\in \mathbb R^n: f(x^*+\alpha d)<f(x^*)\text{ for all } \alpha\in (0,\delta) \text{ for some } \delta>0\}$$
is called the cone of descent directions.

**Definition:** The set $F\cap D$ is called the cone of feasible descent directions.


![alt text](images/feasible_descent_directions.svg "Feasible descent directions")

**(Obvious) Theorem:** Consider an optimization problem 
$$
\begin{align}
\min &\  f(x)\\
\text{s.t. }&\ x\in S
\end{align}
$$
and let $x^*\in S$. Now if $x^*$ is a local minimizer **then** the set of feasible descent directions $F\cap D$ is empty.

## Idea for the methods of feasible descent directions

1. Find a feasible solution $x_0$ as the starting point ($k=0$).
2. Find a feasible descent direction $d_k\in D\cap F$.
3. Determine the step length ($\alpha_k$) to the direction $d_k$ (Use line search to find an optimal step length).
4. Update $x$ accordingly ($x_{k+1} = x_k + \alpha_k d_k$).
5. Check convegency. If not converged, set $k = k+1$ and go to 2.

# Rosen's projected gradient method

Assume a problem with linear equality constraints

$$
\min f(x)\\
\text{s.t. } H(x)=Ax-b=0,
$$

where $A$ is an $l\times n$ matrix ($l\leq n$) and $b$ is a vector.

Let $\mathbf{x}$ be a feasible solution to the above problem.

It holds that:

$$
\mathbf{Ax}=b \\
\rightarrow \mathbf{A}(\mathbf{x} + \alpha \mathbf{d}) = b \\
\rightarrow \mathbf{Ax} + \alpha \mathbf{Ad} = b \\
\rightarrow b + \alpha \mathbf{Ad} = b
$$

Then, $\mathbf{d}$ is a feasible direction *if and only if* $\mathbf{Ad}=0$

Thus, the gradient $-\nabla f(x)$ is a feasible descent direction, if 

$$ A\nabla f(x)=0.$$

This may or may not be true (i.e., the gradient may or may not be a feasible descent direction).

However, we can project the gradient to the set of feasible descent directions
$$ \{d\in \mathbb R^n: Ad=0\},$$
which now is a linear subspace.

![alt text](images/subspace.svg "A linear subspace Ad=0")

### Projection

Let $a\in \mathbb R^n$ be a vector and let $L$ be a linear subspace of $\mathbb R^n$. Now, the following are equivalent
* $a^P$ is the projection of $a$ on $L$,
* $\{a^P\} = \operatorname{argmin}_{l\in L}\|a-l\|$, 
(i.e.,  $a^P$, on $L$, has the minimum distance to $a$) and


![](images/projection.jpg )

## Projected gradient

The projection of the gradient $\nabla f(x)$ on the set $\{d\in \mathbb R^n: Ad=0\}$ is denoted by $\nabla f(x)^P$ and called the *projected gradient*. 

Now, given some conditions, the projected gradient gives us a feasible descent direction.

![alt text](images/projected_gradient.svg "A projected gradient")

![](images/RosenGrad.jpg)

## How to compute the projected gradient?

There are different ways, but at this course we can use optimization. Basically, the optimization problem that we have to solve is
$$
\min \|\nabla f(x)-d\|\\
\text{s.t. }Ad=0.
$$

Since it is equivalent to minimize the square of the objective function $\sum_{i=n}\nabla_i f(x)^2+d_i^2-2\nabla_i f(x)d_i$, we can see that the problem is a quadratic problem with equality constraints,
$$
\min \frac12 d^TId-\nabla f(x)^Td\\
\text{s.t. }Ad=0
$$
which means that we just need to solve the system of equations (see e.g., https://en.wikipedia.org/wiki/Quadratic_programming#Equality_constraints)

$$
\left[
\begin{array}{cc}
I&A^T\\
A&0
\end{array}
\right] 
\left[\begin{align}d\\\lambda\end{align}\right]
= \left[ 
\begin{array}{c}
\nabla f(x)\\
0
\end{array}
\right],
$$

where $I$ is the identity matrix, and $\lambda$ are the Lagrange multipliers.

### Code in Python

#### A function for projecting a vector to a linear space defined by $Ax=0$.

In [2]:
import numpy as np
#help(np.linalg.solve)

In [3]:
import numpy as np
def project_vector(A,vector):
    #convert A into a matrix
    A_matrix = np.matrix(A)
    #construct the "first row" of the matrix [[I,A^T],[A,0]]
    left_matrix_first_row = np.concatenate((np.identity(len(vector)),A_matrix.transpose()), axis=1)
    #construct the "second row" of the matrix
    left_matrix_second_row = np.concatenate((A_matrix,np.matrix(np.zeros([len(A),len(A)]))), axis=1)
    #combine the whole matrix by combining the rows
    left_matrix = np.concatenate((left_matrix_first_row,left_matrix_second_row),axis = 0)
    #Solve the system of linear equalities from the previous page
    return np.linalg.solve(left_matrix, \
                           np.concatenate((np.matrix(vector).transpose(),\
                                           np.zeros([len(A),1])),axis=0))[:len(vector)] 


In [4]:
# Example: Project gradient such that A*proj_gradient = 0
A = [[1,0,0],[0,1,0]]
gradient = [1,1,1]
project_vector(A,gradient)

matrix([[0.],
        [0.],
        [1.]])

# Example

Let us study optimization problem
$$
\begin{align}
\min \qquad& x_1^2+x_2^2+x_3^2\\
\text{s.t.}\qquad &x_1+x_2=3\\
    &x_1+x_3=4.
\end{align}
$$

Let us project a negative gradient from a feasible point $x=(1,2,3)$

Now, the matrix
$$
A = \left[
\begin{array}{ccc}
1& 1 & 0\\
1& 0 & 1
\end{array}
\right]
$$.

In [5]:
import ad
A = [[1,1,0],[1,0,1]]
gradient = ad.gh(lambda x:x[0]**2+x[1]**2+x[2]**2)[0]([1,2,3])
print(gradient)
d = project_vector(A,[-i for i in gradient])
print(d)

[2.0, 4.0, 6.0]
[[ 2.66666667]
 [-2.66666667]
 [-2.66666667]]


### d is a feasible direction

In [ ]:
np.matrix(A)*d

### d is a descent direction

In [8]:
def f(x):
    return x[0]**2+x[1]**2+x[2]**2
alpha = 0.5
print("Value of f at [1,2,3] is "+str(f([1,2,3])))
x_mod= np.array([1,2,3])+alpha*np.array(d).transpose()[0]
print(x_mod)
print("Value of f at [1,2,3] +alpha*d is "+str(f(x_mod)))
print("Gradient dot product direction (i.e., directional derivative) is " \
      + str(np.matrix(ad.gh(f)[0]([1,2,3])).dot(np.array(d))))

Value of f at [1,2,3] is 14
[2.33333333 0.66666667 1.66666667]
Value of f at [1,2,3] +alpha*d is 8.666666666666668
Gradient dot product direction (i.e., directional derivative) is [[-21.33333333]]


## Finally, the algorithm of the projected gradient

In [9]:
import numpy as np
import ad
def projected_gradient_method(f,A,start,step,precision):
    f_old = float('Inf')
    x = np.array(start)
    steps = []
    f_new = f(x)
    iters = 0
    while abs(f_old-f_new)>precision:
        # store the current function value
        f_old = f_new
        # compute gradient
        gradient = ad.gh(f)[0](x)
        # project negative gradient
        d = project_vector(A,[-i for i in gradient])
        # take transpose
        d = d.reshape(1,-1)
        # take step
        x = np.array(x + step*d)[0]
        # compute f in new point+ 
        f_new = f(x)
        # record new step
        steps.append(x)
        # update iterations counter
        iters = iters + 1
    return x,f_new,steps,iters

In [16]:
f = lambda x:x[0]**2+x[1]**2+x[2]**2
A = [[1,1,0],[1,0,1]]
start = [1,2,3]
(x,f_val,steps,iters) = projected_gradient_method(f,A,start,0.5,0.000001)

In [17]:
print(x)
print(f(x))
print(f([1,2,3]))
print(np.matrix(A)*np.matrix(x).transpose())
print(iters)

[2.33333333 0.66666667 1.66666667]
8.666666666666668
14
[[3.]
 [4.]]
2


## Note
If there are both linear equality and inequality constraints, the projection matrix does not remain the same 
* At each iteration, it includes only the equality and active inequality constraints

# Active set method
Consider a problem
$$
\min f(x)\\
\text{s.t. }Ax\leq b,
$$
where $A$ is an $l\times n$ matrix ($l\leq n$) and $b$ is a vector.

## Idea
* In $𝑥_k$, the set of constraints is divided into active ($𝑖 ∈ 𝐼$) and inactive constraints
* Inactive constraints are not taken into account when the search direction $𝑑_k$ is determined
* Inactive constraints affect only when computing the optimal step length $\alpha_k$ (making sure we are not step out of the feasible region)


## Feasible directions
* For $𝑖\in 𝐼$ (active constraints), $(𝑎_i)^Tx_k = b_i$
* If $𝑑_k$ is feasible in $𝑥_k$, then $𝑥_k + \alpha 𝑑_k \in 𝑆$ for some $\alpha > 0$
* $(𝑎_i)^T(x_k+\alpha d_k) = (a_i)^Tx_k + \alpha(a_i)^Td_k\leq b_i$
* $(𝑎_i)^Td_k\leq 0$ for feasible $𝑑_k$ and the constraint remains active if $(𝑎_i)^Td_k=0$

## On active constraints
* Optimization problem with inequality constraints is more difficult than problem with equality constraints since the active set in a local minimizer is not known
* If it would be known, then it would be enough to solve a corresponding equality constrained problem
* In that case, if the other constraints would be satisfied in the solution and all the Lagrange multipliers were non-negative, then the solution would also be a solution to the original problem

## Using active set
* At each iteration, a working set is considered which consists of the active constraints in $𝑥_k$
* The direction $𝑑_k$ is determined so that it is a descent direction in the working set
  * E.g., Rosen’s projected gradient method can be used

## Active set algorithm
1. Choose a starting point $x_1$ and determine an initial active set $I_1$ and set $k=1$
2. Compute a feasible descent direction $d_k$ in the subspace defined by the active constraints (e.g., by using projected gradient)
3. If $||d_k||=0$, go to step 6, otherwise, find optimal step length $\alpha$ by staying in the feasible set and set $x_{k+1} = x_k + \alpha d_k$
4. If no new constraint becomes active go to step 7 (active set does not change)
5. Addition to active set: a new constraint $j$ becomes active, update $I_{k+1} = I_k \cup \{j\}$ and go to step 7
6. Removal from active set: approximate Lagrangian multipliers $\mu_i$, $i\in I_k$. If $\mu_i\geq 0$ for all $i$, stop (active set is correct). Otherwise, remove a constraint $j$ with negative multiplier from the active set: $I_{k+1}=I_k\setminus \{j\}$
7. Set $k=k+1$ and go to step 2

<i>Implementation of the active set method is left as a voluntary exercise </i>

### Note: 

* Projected gradient method can also extended for non-linear constraints.
* But, this needs some extra steps

![](Images/project-nl.jpg)